In [ ]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D,GlobalAveragePooling1D
from tensorflow.keras import Model, layers,Sequential,regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
from utils import get_train_data,get_train_split,get_test_split,preprocess

In [ ]:
X_train,y_train =  get_train_split()
X_test,y_test = get_test_split()

In [ ]:
X_train_processed, y_train_processed, X_test_ = preprocess(X_train,y_train,X_test)

In [ ]:
y_train_processed = to_categorical(y_train_processed, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [ ]:
# Define your model architecture
model= Sequential()
model.add(Convolution1D(32,5,activation='relu',input_shape=(29,1)))
model.add(Convolution1D(64,5,activation='relu'))         
model.add(MaxPooling1D(3))
model.add(Convolution1D(128, 3, activation='relu'))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.3))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation='softmax'))

# Compile the model
learning_rate = 0.001  # Set the desired learning rate

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


# Define early stopping callback
early_stopping = EarlyStopping(patience=3, monitor='val_loss')

# Train the model with early stopping
trained_model = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)

# Make predictions on new data
train_predictions = model.predict(X_test)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the hyperparameters to tune
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
num_epochs = [5, 10]

best_accuracy = 0.0
best_model_path = None
best_run_id = None

def train_model(learning_rate, batch_size, num_epochs):

    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)

    global best_accuracy
    global best_model_path
    global best_run_id
    
    # Define the model architecture
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    early_stopping = EarlyStopping(patience=3, monitor='val_loss')
    history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val), 
                        callbacks=[early_stopping],verbose=1)
    
    # Evaluate the model on the validation set
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    
    # Log the validation loss and accuracy
    mlflow.log_metric("val_loss", val_loss)
    mlflow.log_metric("val_accuracy", val_accuracy)
    
    # Check if the current model is the best so far
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        
        # Save the model
        best_model_path = f"best_model_lr_{learning_rate}_bs_{batch_size}_epochs_{num_epochs}.h5"
        model.save(best_model_path)
        
        # Get the run ID for the best model
        best_run_id = mlflow.active_run().info.run_id
    
    return model

# Iterate over the hyperparameters
for lr in learning_rates:
    for bs in batch_sizes:
        for epochs in num_epochs:
            # Start a new MLflow run
            with mlflow.start_run(run_name=f"lr_{lr}_bs_{bs}_epochs_{epochs}"):
                # Train the model
                model = train_model(lr, bs, epochs)
                
                # # Log the hyperparameters only once per run
                # if epochs == num_epochs[0]:
                #     mlflow.log_param("learning_rate", lr)
                #     mlflow.log_param("batch_size", bs)
                #     mlflow.log_param("num_epochs", epochs)

# Load the best model
best_model = tf.keras.models.load_model(best_model_path)

# Print the run ID for the best model
print(f"Best Run ID: {best_run_id}")

In [ ]:
# Retrieve the run information for the best run
run_info = mlflow.get_run(best_run_id)

# Get the hyperparameters logged for the best run
hyperparams = run_info.data.params

if 'learning_rate' in hyperparams:
    print(f"Learning Rate: {hyperparams['learning_rate']}")

if 'batch_size' in hyperparams:
    print(f"Batch Size: {hyperparams['batch_size']}")

if 'num_epochs' in hyperparams:
    print(f"Number of Epochs: {hyperparams['num_epochs']}")

In [ ]:
# Evaluate the model on test data
loss, accuracy = best_model.evaluate(X_test, y_test)

# Use the best model for predictions or other tasks
predictions = best_model.predict(X_test)

In [ ]:
# Convert the predicted labels to class indices (assuming one-hot encoding)
predicted_indices = tf.argmax(predictions, axis=1).numpy()

# Convert the actual labels to class indices (assuming one-hot encoding)
actual_indices = tf.argmax(y_test, axis=1).numpy()

# Calculate the accuracy
accuracy = accuracy_score(actual_indices, predicted_indices)

print("Accuracy:", accuracy)


In [ ]:
fig1, ax_acc = plt.subplots()
plt.plot(trained_model.history['accuracy'])
plt.plot(trained_model.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model - Accuracy')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()

In [ ]:
fig2, ax_loss = plt.subplots()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model- Loss')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.plot(trained_model.history['loss'])
plt.plot(trained_model.history['val_loss'])
plt.show()
target_names=['0','1','2','3','4']

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(actual_indices, predicted_indices)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
n_holdout_samples = 1000  # Creating 1000 new samples

# Create a holdout set by randomly sampling from the training dataset
holdout_samples_indices = np.random.choice(len(x), size=n_holdout_samples, replace=False)
X_holdout = X_train[holdout_samples_indices]
y_holdout = y_train[holdout_samples_indices]

In [ ]:
loss, accuracy = model.evaluate(X_holdout, y_holdout)
loss, accuracy = best_model.evaluate(X_holdout, y_holdout)